# Risk & Leverage vs Sentiment Analysis

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Set style and color palette
plt.style.use('seaborn-darkgrid')
sns.set_palette('husl')

# Read the merged data
df = pd.read_csv('merged_data.csv')

# Display first few rows
df.head()

## Analyze Position Size by Market Sentiment

In [ ]:
# Create sentiment categories
df['Sentiment_Category'] = pd.cut(df['value'],
                                  bins=[0, 25, 50, 75, 100],
                                  labels=['Extreme Fear', 'Fear', 'Neutral', 'Greed'])

# Calculate average position size by sentiment
avg_position_size = df.groupby('Sentiment_Category')['Size USD'].mean().reset_index()

# Create enhanced bar plot
plt.figure(figsize=(12, 6))

# Create bar plot with gradient colors
bars = plt.bar(avg_position_size['Sentiment_Category'], avg_position_size['Size USD'])

# Create color gradient
colors = plt.cm.RdYlBu(np.linspace(0, 1, len(bars)))
for bar, color in zip(bars, colors):
    bar.set_color(color)

# Customize plot
plt.title('Average Position Size by Market Sentiment', fontsize=14, pad=20)
plt.xlabel('Market Sentiment', fontsize=12)
plt.ylabel('Average Position Size (USD)', fontsize=12)
plt.xticks(rotation=45)

# Add value labels on top of bars
for i, v in enumerate(avg_position_size['Size USD']):
    plt.text(i, v, f'${v:,.0f}', ha='center', va='bottom')

# Add grid for better readability
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Calculate Daily Risk Metrics

In [ ]:
# Convert timestamp to datetime
df['Date'] = pd.to_datetime(df['Timestamp'])

# Calculate daily risk metrics
daily_metrics = df.groupby('Date').agg({
    'Size USD': ['mean', 'std'],
    'PnL': ['mean', 'std'],
    'value': 'first'
}).reset_index()

# Flatten column names
daily_metrics.columns = ['Date', 'Avg_Position_Size', 'Position_Size_Std',
                        'Avg_PnL', 'PnL_Std', 'Fear_Greed_Index']

# Calculate risk-adjusted returns (Sharpe-like ratio)
daily_metrics['Risk_Adjusted_Returns'] = daily_metrics['Avg_PnL'] / daily_metrics['PnL_Std'].fillna(1)

# Create enhanced scatter plot
plt.figure(figsize=(12, 6))

# Create scatter plot with size based on position size
scatter = plt.scatter(daily_metrics['Fear_Greed_Index'],
                     daily_metrics['Risk_Adjusted_Returns'],
                     c=daily_metrics['Avg_Position_Size'],
                     s=100, cmap='viridis', alpha=0.6)

# Add trend line
z = np.polyfit(daily_metrics['Fear_Greed_Index'],
              daily_metrics['Risk_Adjusted_Returns'], 1)
p = np.poly1d(z)
plt.plot(daily_metrics['Fear_Greed_Index'],
         p(daily_metrics['Fear_Greed_Index']),
         'r--', alpha=0.8)

# Customize plot
plt.title('Risk-Adjusted Returns vs Fear & Greed Index', fontsize=14, pad=20)
plt.xlabel('Fear & Greed Index', fontsize=12)
plt.ylabel('Risk-Adjusted Returns', fontsize=12)

# Add colorbar
cbar = plt.colorbar(scatter)
cbar.set_label('Average Position Size (USD)', fontsize=10)

# Add grid for better readability
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Risk Metrics Distribution

In [ ]:
# Create sentiment categories for daily metrics
daily_metrics['Sentiment_Category'] = pd.cut(daily_metrics['Fear_Greed_Index'],
                                             bins=[0, 25, 50, 75, 100],
                                             labels=['Extreme Fear', 'Fear', 'Neutral', 'Greed'])

# Create subplots for risk metrics distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot position size distribution
sns.violinplot(data=daily_metrics, x='Sentiment_Category', y='Avg_Position_Size',
               palette='RdYlBu', inner='box', ax=ax1)
ax1.set_title('Position Size Distribution by Sentiment', fontsize=12)
ax1.set_xlabel('Market Sentiment', fontsize=10)
ax1.set_ylabel('Average Position Size (USD)', fontsize=10)
ax1.tick_params(axis='x', rotation=45)

# Plot risk-adjusted returns distribution
sns.violinplot(data=daily_metrics, x='Sentiment_Category', y='Risk_Adjusted_Returns',
               palette='RdYlBu', inner='box', ax=ax2)
ax2.set_title('Risk-Adjusted Returns Distribution by Sentiment', fontsize=12)
ax2.set_xlabel('Market Sentiment', fontsize=10)
ax2.set_ylabel('Risk-Adjusted Returns', fontsize=10)
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Calculate and display correlation
correlation = daily_metrics['Fear_Greed_Index'].corr(daily_metrics['Risk_Adjusted_Returns'])
print(f'
Correlation between Risk-Adjusted Returns and Fear & Greed Index: {correlation:.2f}')

# Calculate summary statistics
summary_stats = daily_metrics.groupby('Sentiment_Category').agg({
    'Avg_Position_Size': ['mean', 'std'],
    'Risk_Adjusted_Returns': ['mean', 'std']
}).round(2)

print('
Summary Statistics by Sentiment Category:')
print(summary_stats)